In [4]:
import pandas as pd
import numpy as np
import os

from itertools import product

import Finalized_pipeline

import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
main_datasets = ["BACE_class", "BACE_regre", "ROR_class", "ROR_regre"]
model_name_shorts = {"DecisionTreeRegressor": "dt", "DecisionTreeClassifier": "dt", 
                    "RandomForestRegressor": "rf", "RandomForestClassifier": "rf", 
                    "LinearRegression": "lr", "LogisticRegression": "lr", 
                    "MLPRegressor": "nn", "MLPClassifier": "nn",  
                    "GradientBoostingRegressor": "gb", "GradientBoostingClassifier": "gb",
                    "XGBRegressor": "xg", "XGBClassifier": "xg", 
                    "SVR": "sv", "SVC": "sv"}
    
top_models = 100


In [6]:
for dataset in main_datasets:
    regression = False
    if "regre" in dataset:
        regression = True


    df = pd.read_csv(r"Merged runs/" + dataset + ".csv")
    
    setups = {"Fingerprints": [], "Descriptors": [], "DescFing": []}

    for row in df.head(top_models)[['hyperparams', 'preprocessing', 'model']].iterrows():
        #print(row)
        params = list(row[1])[0].split(", '")
        params[0] = params[0][2:]
        params[-1] = params[-1][:-1]
        #print(params)

        preprocessing = list(row[1])[1]
        model = list(row[1])[2]
        #print(preprocessing)

        clean_params = {}
        
        for param in params:
            name = param.split("': ")[0]
            val = param.split("': ")[1]
            if val[0] == "'":
                val = val[1:-1]
            elif val[0] == "(":
                if val == "(50, 50)": val = (50, 50)
                if val == "(100, )" or val == "(100,)": val = (100, )
                if val == "(50, )" or val == "(50,)": val = (50, )
            elif val == "None":
                val = None
            else:
                try:
                    val = int(val)
                except:
                    val = float(val)
            clean_params[name] = [val]
        #print(model)
        #print(preprocessing)
        #print(clean_params)
        setups[preprocessing].append([model, clean_params])

    for key in setups.keys():
        setups[key] = sorted(setups[key], key=lambda x:x[0])
    #print(setups)

    returned_df = pd.DataFrame(columns=df.columns)
    #print(returned_df)

    for key in setups.keys():
        if key == "Descriptors":
            calculate_descriptors = True
            calculate_fingerprints = False
        if name == "DescFing":
            calculate_descriptors = True
            calculate_fingerprints = True
        if key == "Fingerprints":
            calculate_descriptors = False
            calculate_fingerprints = True

            
        if "BACE" in dataset:
            prepared_dataset = pd.read_csv(r"C:\Users\wojci\Documents\GitHub\czasteczkowa-inzynierka\experiments\split_datasets\split0.9_bace.csv")
        else:
            prepared_dataset = pd.read_csv(r"C:\Users\wojci\Documents\GitHub\czasteczkowa-inzynierka\experiments\split_datasets\split0.9_ROR_data_1.csv")
        
        if not "pIC50" in prepared_dataset.columns:
            prepared_dataset = Finalized_pipeline.calculate_pIC50(prepared_dataset, "target")
        
        if not regression:
            prepared_dataset = Finalized_pipeline.calculate_classification_labels(prepared_dataset, "pIC50", threshold=7)
            target_column = "label"
        else:
            target_column = "pIC50"

        if "mol" in prepared_dataset.columns:
            prepared_dataset.rename(columns={"mol": "SMILES"}, inplace=True)

        if regression:
            runtype = "regression"
        else:
            runtype = "classification"
        
        prepared_dataset = Finalized_pipeline.calculate_features(prepared_dataset, calculate_descriptors=calculate_descriptors, 
                                                                 calculate_fingerprints=calculate_fingerprints, 
                                                    SMILES_column_name="SMILES", target_column_name=target_column, 
                                                    split_column_name="Split", output_csv_path=dataset + "_" + key + "_0.9.csv")

        for combination in setups[key]:
                            # model short name, parameters
            hyperparams = {str(model_name_shorts[combination[0]]): combination[1]}


            new_df = Finalized_pipeline.hyperparameter_search(prepared_dataset, hyperparams, unique=False, return_df=True, output_file_name="tmp.csv")

            new_df['split'] = 0.9
            new_df['preprocessing'] = key

            returned_df = pd.concat([returned_df, new_df])

    if "mse" in returned_df.columns:
        returned_df.sort_values(by=['mse'], inplace=True)
    else:
        returned_df.sort_values(by=['roc_auc'], inplace=True, ascending=False)
    returned_df.to_csv("RERUN_" + dataset + ".csv", index=False)




A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Regression:  False
gb
{'max_depth': 3, 'loss': 'exponential', 'n_estimators': 200, 'learning_rate': 1.0, 'min_impurity_decrease': 0.05}
time:  3.095628
0 0.8139413906710595 0.9895282375959022 0.8157894736842105 0.9897134459955915 0.835820895522388 0.988135593220339 0.7671232876712328 0.988135593220339 0.8 0.988135593220339 GradientBoostingClassifier {'max_depth': 3, 'loss': 'exponential', 'n_estimators': 200, 'learning_rate': 1.0, 'min_impurity_decrease': 0.05}
Regression:  False
gb
{'max_depth': 3, 'loss': 'exponential', 'n_estimators': 100, 'learning_rate': 0.5, 'min_impurity_decrease': 0}
time:  1.887843
0 0.8418588520894745 0.9642441029699489 0.8421052631578947 0.9639970609845702 0.8356164383561644 0.9515859766277128 0.8356164383561644 0.9661016949152542 0.8356164383561644 0.9587888982338099 GradientBoostingClassifier {'max_depth': 3, 'loss': 'exponential', 'n_estimators': 100, 'learning_rate': 0.5, 'min_impurity_decrease': 0}
Regression:  False
gb
{'max_depth': 5, 'loss': 'exponen

The max_iter was reached which means the coef_ did not converge


time:  1.781753
0 0.8613663950060689 0.9560641034096155 0.8618421052631579 0.9551800146950772 0.8611111111111112 0.9357495881383855 0.8493150684931506 0.9627118644067797 0.8551724137931035 0.9490392648287386 LogisticRegression {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Regression:  False
lr
{'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
time:  0.0917
0 0.8613663950060689 0.9569115610367341 0.8618421052631579 0.9559147685525349 0.8611111111111112 0.9358552631578947 0.8493150684931506 0.964406779661017 0.8551724137931035 0.9499165275459098 LogisticRegression {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Regression:  False
lr
{'C': 10, 'penalty': 'l2', 'solver': 'saga'}


The max_iter was reached which means the coef_ did not converge


...

time:  15.024897
0 0.6735705691993629 0.5912508684313695 0.45369731169155375 0.3495775894208487 0.5334175983160212 0.45890308746954156 0.49712759397312645 0.6213010857398049 MLPRegressor {'hidden_layer_sizes': (50,), 'activation': 'tanh', 'alpha': 0.01, 'max_iter': 200, 'solver': 'sgd'}
Regression:  True
nn
{'hidden_layer_sizes': (50,), 'activation': 'logistic', 'alpha': 0.01, 'max_iter': 1000, 'solver': 'sgd'}
time:  64.01071
0 0.6714924300480766 0.570179589938508 0.450902083611871 0.32510476478244515 0.5366418823444086 0.440925315924969 0.5002257896943731 0.6478126024957782 MLPRegressor {'hidden_layer_sizes': (50,), 'activation': 'logistic', 'alpha': 0.01, 'max_iter': 1000, 'solver': 'sgd'}
Regression:  True
nn
{'hidden_layer_sizes': (50,), 'activation': 'logistic', 'alpha': 0.0001, 'max_iter': 1000, 'solver': 'sgd'}
time:  63.915106
0 0.6714866390546593 0.5699285303350572 0.4508943064289223 0.3248185296898782 0.5366376470371638 0.440709455613924 0.5002344098263439 0.6481226822092915


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 2186/2186 [06:53<00:00,  5.28it/s]


Regression:  True
rf
{'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'min_impurity_decrease': 0, 'n_estimators': 50}
time:  67.748587
0 0.6510687131929452 0.42899737258502957 0.42389046929871765 0.1840387456848587 0.5019026933553127 0.3320013171515713 0.5301651239824283 0.8006300309807344 RandomForestRegressor {'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'min_impurity_decrease': 0, 'n_estimators': 50}
Regression:  True
rf
{'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0, 'n_estimators': 100}
time:  142.485232
0 0.6505145220421026 0.4060940873841225 0.4231691433876652 0.16491240780834332 0.4978103077197902 0.3169562299307548 0.530964632569041 0.8213496755083194 RandomForestRegressor {'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1, 'min_impurity_decrease': 0, 'n_estimators': 100}



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 2186/2186 [07:01<00:00,  5.19it/s]


Regression:  True
gb
{'max_depth': 7, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.2}
time:  101.403883
0 0.6375208169234879 0.25634766270736253 0.4064327920107914 0.0657141241755277 0.4723842972195164 0.17247839968849005 0.5495149943810183 0.9288116051201657 GradientBoostingRegressor {'max_depth': 7, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.2}
Regression:  True
gb
{'max_depth': 7, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.1}
time:  127.901451
0 0.6326613315061276 0.2249507378828789 0.40026036038310625 0.050602834474051686 0.4716272485289933 0.1433573006905876 0.5563564401283552 0.9451817305978916 GradientBoostingRegressor {'max_depth': 7, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.1}
Regression:  True
gb
{'max_depth': 5, 'loss': 'huber', 'n_estimators': 100, 'learning_rate': 0.1, 'min_impurity_decrease': 0.1}
t